In [1]:
from IPython.display import Image

- 输入是 state，输出是各 action 对应的 `Q(s,a)`
    - Model-free
- 如何转换为 policy，应用到交互中
    -  ε-greedy (epsilon-greedy)
 
$$
a = \arg\max(Q(s, a))
$$

### DVN? => Actor-Critic

In [2]:
Image(url='http://incompleteideas.net/book/ebook/figtmp34.png', width=200)

- 假如 DVN 学到的是 $V(s)$
    - 它不直接提供任何关于“哪个动作更好”的信息。为了做出决策，你必须能够“向前看一步”，弄清楚每个动作会把你带到哪个新状态 s'，然后比较那些新状态的价值 V(s')。
    - 这个“**向前看**”的过程需要一个环境模型 (Environment Model，model-based)。这个模型需要能回答这个问题：T(s, a) -> s'，即在状态 s 执行动作 a，下一个状态 s' 是什么？

$$
a = \arg\max_a [ R(s,a) + γ * V(s') ] 
$$

- DVN 不像 DQN 可以独立存在，而是跟 policy network 一起 work，所谓的 Actor-Critic
    - $A(s,a) ≈ r + γV(s') - V(s)$
        - “在状态 s 下，执行动作 a 到底比**通常情况（平均水平）**好多少？”
    - 学习 V(s) 的 Critic 网络不直接用于决策，而是用于指导 Actor 网络进行更有效的学习。这结合了 DQN (价值学习) 和策略梯度 (直接学习策略) 的优点。像 A2C、A3C、PPO 等都是非常成功的 Actor-Critic 算法。
- actor-critic
    - critic：价值网络($v(s)$)，如果仅有价值网络（策略网络？？），容易贪心，陷入局部左右值；
        - critic 单步打分，reward model：最终裁决；
        - critic 网络的评价（estimated advantage，优势函数）提供了 actor 的更新方向；
        - 优势函数：实际期望 - 预期期望（评价家预测）
            - 大于 0，这个策略就值得鼓励；
    - actor：策略网络($\pi(a|s)$)，概率输出，增加探索的可能；
    - 两者都需要不断训练；
- 仅有 actor 模型（REINFORCE）的问题
    - 高方差 (High Variance)：学习信号极其不稳定，导致训练过程像坐过山车，收敛缓慢且困难。
    - 样本效率低 (Low Sample Efficiency)：必须等待整个回合（episode）结束后才能进行学习，浪费了大量经验。
        - 信用分配不当和学习缓慢